In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [3]:
url <- "https://raw.githubusercontent.com/yxing6/DSCI_100_Project_Group4/main/data/of_8460_database.csv"
download.file(url, destfile = "rock_physical_properties.csv")
rock_data_all <- read_csv("rock_physical_properties.csv")
glimpse(rock_data_all)

Parsed with column specification:
cols(
  .default = col_character(),
  `Date Mapped (yyyy/mm/dd) *` = col_double(),
  `Lat Deg` = col_double(),
  `Long Deg` = col_double(),
  `UTM zone` = col_double(),
  Easting = col_double(),
  Northing = col_double(),
  `Method Precision (m)*` = col_double(),
  `Year *` = col_double(),
  `Lab Date` = col_double(),
  `GRAIN DEN Sample Value [g/cm3]` = col_double(),
  `DRY BULK DEN Sample Value [g/cm3]` = col_double(),
  `SAT BULK DEN Sample Value [g/cm3]` = col_double(),
  `POR Sample Value [%]` = col_double(),
  `MS Sample Value [SI A/m / A/m]` = col_double(),
  `NRM Sample Value [A/m]` = col_double(),
  `KN50 Sample Value` = col_double(),
  `Local Field [uT]` = col_double(),
  KN = col_double(),
  `RES Sample Value [Ohm.m]` = col_double(),
  `CHG Sample Value [ms]` = col_double()
)

See spec(...) for full column specifications.



Rows: 19,653
Columns: 84
$ `0Phys Props LabID`                          <chr> "BC13148", "BC13149", "B…
$ `Sample Name *`                              <chr> "MDM 501", "MDM 502", "M…
$ `Alt Sample Name`                            <chr> NA, NA, NA, NA, NA, NA, …
$ `SMS Curation Number`                        <chr> NA, NA, NA, NA, NA, NA, …
$ `Previous Mira RPDS ID`                      <chr> "MDM 501", "MDM 502", "M…
$ `StationName/Well ID/BoreholeName *`         <chr> "MDM 501", "MDM 502", "M…
$ `Location area`                              <chr> "6 BC Porphyries", "6 BC…
$ `Location Description`                       <chr> NA, NA, NA, NA, NA, NA, …
$ `Mapped By *`                                <chr> "Mitchinson D", "Mitchin…
$ `Date Mapped (yyyy/mm/dd) *`                 <dbl> 2009, 2009, 2009, 2009, …
$ `Location Type *`                            <chr> "Borehole", "Borehole", …
$ `Country *`                                  <chr> "Canada", "Canada", "Can…
$ `Province/ Territory (* i

In [4]:
rock_data_selected <- select(rock_data_all, 
    "Location Type *", "MIRA Master Litho 1", "GRAIN DEN Sample Value [g/cm3]",
    "POR Sample Value [%]","MS Sample Value [SI A/m / A/m]",
    "NRM Sample Value [A/m]","RES Sample Value [Ohm.m]","CHG Sample Value [ms]") %>%
    rename("sample_type" = "Location Type *",
          "lithology" = "MIRA Master Litho 1",
           "density" = "GRAIN DEN Sample Value [g/cm3]",
           "porosity" = "POR Sample Value [%]",
           "MS" = "MS Sample Value [SI A/m / A/m]",
           "NRM" = "NRM Sample Value [A/m]",
           "RES" = "RES Sample Value [Ohm.m]",
           "chargeability" = "CHG Sample Value [ms]") %>%
    mutate(sample_type = as_factor(sample_type),
          lithology = as_factor(lithology))
rock_data_selected

borehole_data <- filter(rock_data_selected, sample_type == "Borehole")
borehole_data

sample_type,lithology,density,porosity,MS,NRM,RES,chargeability
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Borehole,Igneous,2.931,0.41,0.00118,0.029,7960,46.4
Borehole,Igneous,2.770,0.28,0.07900,4.810,135000,19.5
Borehole,Igneous,2.754,0.28,0.01260,0.352,51800,1.6
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Outcrop,Igneous,2.679,1.91,0.01240,0.0244,NA,NA
Outcrop,Igneous,2.665,1.56,0.01500,0.0678,NA,NA
Subcrop,Igneous,2.261,9.03,0.00039,NA,NA,NA


sample_type,lithology,density,porosity,MS,NRM,RES,chargeability
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Borehole,Igneous,2.931,0.41,0.00118,0.029,7960,46.4
Borehole,Igneous,2.770,0.28,0.07900,4.810,135000,19.5
Borehole,Igneous,2.754,0.28,0.01260,0.352,51800,1.6
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Borehole,Metamorphic,2.665,0.08,NA,NA,91800,3.868
Borehole,Metamorphic,2.641,0.09,NA,NA,67800,1.388
Borehole,Metamorphic,2.811,0.36,0.0739,5.72,437000,12.850


In [5]:
#density[g/cm3]
#porosity[%]
#MS[m3/kg]
#NRM[A/m]
#RES[Ohm.m]
#chargeability[ms]


In [6]:
data_split <- initial_split(borehole_data, prop = 0.75, strata = lithology)
data_train <- training(data_split)
data_test <- testing(data_split)

In [8]:
glimpse(data_train)
glimpse(data_test)

Rows: 1,064
Columns: 8
$ sample_type   <fct> Borehole, Borehole, Borehole, Borehole, Borehole, Boreh…
$ lithology     <fct> Igneous, Igneous, Igneous, Igneous, Igneous, Igneous, I…
$ density       <dbl> 2.931, 2.770, 2.615, 2.804, 2.852, 2.712, 3.103, 2.864,…
$ porosity      <dbl> 0.41, 0.28, 0.72, 0.34, 0.65, 0.33, 0.91, 0.41, 0.62, 0…
$ MS            <dbl> 1.18e-03, 7.90e-02, 6.56e-05, 1.78e-01, 2.76e-01, 3.79e…
$ NRM           <dbl> 0.029000, 4.810000, 0.000648, 2.980000, 3.820000, 0.593…
$ RES           <dbl> 7960, 135000, 5250, 17200, 6400, 56200, 47, 13500, 3180…
$ chargeability <dbl> 46.400, 19.500, 2.500, 12.600, 9.500, 4.700, 18.400, 9.…
Rows: 353
Columns: 8
$ sample_type   <fct> Borehole, Borehole, Borehole, Borehole, Borehole, Boreh…
$ lithology     <fct> Igneous, Igneous, Igneous, Igneous, Igneous, Igneous, I…
$ density       <dbl> 2.754, 2.716, 2.744, 2.932, 2.938, 2.868, 2.636, 2.696,…
$ porosity      <dbl> 0.28, 0.28, 0.33, 0.59, 0.34, 0.74, 0.86, 0.55, 0.45, 2…
$ MS    

In [32]:
training_data_prop <- data_train %>%
    group_by(lithology) %>%
    summarize(n = n(), 
              across(density:chargeability, mean, na.rm = TRUE))

training_data_prop

`summarise()` ungrouping output (override with `.groups` argument)



lithology,n,density,porosity,MS,NRM,RES,chargeability
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Igneous,254,2.807596,2.3849402,0.0206071624,1.40110729,843159.793,11.436859
Metamorphic,638,2.770426,0.5061164,0.0017773077,0.47438252,7662.322,5.973305
Sedimentary,159,2.660220,6.6002516,0.0002486715,0.02507703,3754.907,4.322911
Other,13,3.452308,0.3107692,0.0017542154,4.53268790,12520.483,32.553538


In [36]:
jjj <- data_train %>%
    group_by(lithology) %>%
    summarize(across(density:chargeability, count, na.rm = FALSE)

jjj

ERROR: Error in parse(text = x, srcfile = src): <text>:5:1: unexpected symbol
4: 
5: jjj
   ^
